In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from preprocess import preprocess, categories_id
from features import outcomes, TRACE_TYPE, IS_XSRC, IS_YSRC, FID, FIELD_ID
import numpy as np

In [ ]:
transform = lambda x: categories_id[(x[TRACE_TYPE], x[IS_XSRC], x[IS_YSRC])]

def dummy_folds(file_id,  n_folds, data_directory='./base', output_directory='./folds'):

    dataset_file = f'{data_directory}/dummy_dataset_{file_id}.csv'
    dataset_df = preprocess(dataset_file)
    dataset_df['class'] = dataset_df[outcomes].apply(transform, axis=1)
    dataset_df = dataset_df.drop(outcomes, axis=1)
    datasets = np.array(list(set(dataset_df[FID])))
    kf = KFold(n_folds)
    folds = []
    fold = 1
    for train_index, validate_index in kf.split(datasets):
        train_datasets = set(datasets[train_index])
        validate_datasets = set(datasets[validate_index])
        train_df = dataset_df[dataset_df[FID].isin(train_datasets)]
        validate_df = dataset_df[dataset_df[FID].isin(validate_datasets)]
        train_df[['set', 'fold']] = 'T', fold
        validate_df[['set','fold']] = 'V', fold
        train_df = train_df.drop([FID, FIELD_ID], axis=1)
        validate_df = validate_df.drop([FID, FIELD_ID], axis=1)

        folds.append(train_df)
        folds.append(validate_df)
        fold += 1
    
    folds_df = pd.concat(folds)
    output_file = f'{output_directory}/dummy_{file_id}_{n_folds}_folds.csv'
    
    with open(output_file, 'w') as f:
        folds_df.to_csv(f, index=False)

In [ ]:
dummy_folds('1k', 5)
dummy_folds('3k', 5)
dummy_folds('5k', 5)
dummy_folds('10k', 5)